<a href="https://colab.research.google.com/github/NewAnik/DataScience/blob/master/Car_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
#import jovian
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [ ]:
DATA_FILENAME = "Car.csv"
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [ ]:
your_name = "Anik Biswas" # at least 5 characters
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.Year = dataframe.Year * ord(rand_str[1])/100.
    # scale target
    dataframe.Selling_Price = dataframe.Selling_Price * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['Car_Name'], axis=1)
    return dataframe

dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
2,2218.7,7.6125,9.85,6900,Petrol,Dealer,Manual,0
128,2218.7,0.8400,0.87,3000,Petrol,Individual,Manual,0
286,2217.6,5.9325,7.90,28569,Petrol,Dealer,Manual,0
272,2216.5,7.8750,10.00,27600,Petrol,Dealer,Manual,0
205,2217.6,5.5125,5.70,3493,Petrol,Dealer,Manual,1


In [ ]:
input_cols = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_cols = ["Fuel_Type","Seller_Type","Transmission"]
output_cols = ["Selling_Price"]

In [ ]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array


(array([[2.2187e+03, 9.8500e+00, 6.9000e+03, 0.0000e+00],
        [2.2187e+03, 8.7000e-01, 3.0000e+03, 0.0000e+00],
        [2.2176e+03, 7.9000e+00, 2.8569e+04, 0.0000e+00],
        ...,
        [2.2187e+03, 3.6230e+01, 6.0000e+03, 0.0000e+00],
        [2.2088e+03, 5.2000e-01, 5.0000e+05, 0.0000e+00],
        [2.2165e+03, 8.4000e-01, 5.8000e+04, 0.0000e+00]]), array([[ 7.6125],
        [ 0.84  ],
        [ 5.9325],
        [ 7.875 ],
        [ 5.5125],
        [ 0.504 ],
        [ 2.9925],
        [ 0.3675],
        [ 1.8375],
        [ 5.1975],
        [ 9.7125],
        [ 5.565 ],
        [ 4.095 ],
        [ 9.555 ],
        [ 4.62  ],
        [ 3.6645],
        [20.7375],
        [ 7.6125],
        [11.8125],
        [13.545 ],
        [15.4665],
        [ 5.145 ],
        [ 5.5125],
        [ 3.4125],
        [ 7.8225],
        [ 0.5775],
        [ 3.045 ],
        [ 0.3255],
        [ 0.2835],
        [ 7.77  ],
        [ 0.63  ],
        [ 6.4575],
        [ 2.9925],
        [ 8

In [ ]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

dataset = TensorDataset(inputs, targets)
train_ds, val_ds = random_split(dataset, [228, 57])
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [ ]:
input_size = len(input_cols)
output_size = len(output_cols)

class CarsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                         # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))
            
model = CarsModel()

list(model.parameters())

[Parameter containing:
 tensor([[-0.0329,  0.0878, -0.2516,  0.1787]], requires_grad=True),
 Parameter containing:
 tensor([0.2172], requires_grad=True)]

In [ ]:
# Eval algorithm
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

# Fitting algorithm
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

# Check the initial value that val_loss have
result = evaluate(model, val_loader)
print(result)

{'val_loss': 8401.701171875}


In [ ]:
# Start with the Fitting
epochs = 90
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 7891.3481
Epoch [40], val_loss: 7379.8975
Epoch [60], val_loss: 6868.2275
Epoch [80], val_loss: 6355.4390
Epoch [90], val_loss: 6098.7622


In [ ]:
# Train repeatdly until have a 'good' val_loss
epochs = 20
lr = 1e-9
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6047.4170


In [ ]:
# Prediction Algorithm
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

# Testing the model with some samples
input, target = val_ds[0]
predict_single(input, target, model)


Input: tensor([2.2132e+03, 9.4000e+00, 6.0000e+04, 0.0000e+00])
Target: tensor([5.1975])
Prediction: tensor([-10905.5684])


In [ ]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([2.2099e+03, 1.8610e+01, 6.2000e+04, 0.0000e+00])
Target: tensor([3.9900])
Prediction: tensor([-11266.0947])
